## Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
from os import read
import pandas as pd
import math
import random
import seaborn as sns
import time 
# from tsfresh import extract_features, extract_relevant_features, select_features
# from tsfresh.utilities.dataframe_functions import impute
# from tsfresh.feature_extraction import ComprehensiveFCParameters
# from tsfresh.feature_selection.relevance import calculate_relevance_table
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import mutual_info_classif


def checkNotEmpty(input_array):
    for x in input_array:
        if float(x) > 0:
            return True
    return False

def getAverage(input_array):
    acc = 0.0
    for x in input_array:
        acc+=x
    return acc / len(input_array)

def getWindowFrameFromDir(source_dir, window_size, verdict):
    valid = 0
    cache_windows = []
    ipc_windows = []
    ver_windows = []
    for subdir, dirs, files in os.walk(source_dir):
        for dir in dirs:
            cache = []
            ipc = []
            ipc_window = []
            cache_window= []
            for filename in os.listdir(os.path.join(subdir, dir)):
                f = os.path.join(subdir, dir, filename)
                actualfile = open(f,'r')
                if (filename[:3] == "ipc"):
                    ipc = actualfile.readlines()
                else:
                    cache = actualfile.readlines()
            dipc = window_size-len(ipc)
            dcache = window_size-len(cache)
            if (checkNotEmpty(cache) and checkNotEmpty(ipc)):
                for x in range(0, len(ipc)):
                    ipc_window.append(float(ipc[x]))
                for x in range(0, len(cache)):
                    # let's normalize cache access to IPC
                    cache_window.append(float(cache[x])/1000000)
                start = time.time_ns()
                cache_windows.append(getAverage(cache_window))
                ipc_windows.append(getAverage(ipc_window))
                if verdict:
                    ver_windows.append(1)
                end = time.time_ns()
                #print("Average time = " + str((end-start) /1000))
                valid += 1
    print("Total windows = " + str(valid))
    if verdict:
        dataframe = pd.DataFrame(list(zip(ipc_windows, cache_windows, ver_windows)), columns=['avg_ipc', 'avg_cache', 'verdict'])
    else:
        dataframe = pd.DataFrame(list(zip(ipc_windows, cache_windows)), columns=['avg_ipc', 'avg_cache'])
    return dataframe

def getWindowFrameFromDirAttack(source_dir, window_size, verdict):
    valid = 0
    cache_windows = []
    ipc_windows = []
    ver_windows = []
    for subdir, dirs, files in os.walk(source_dir):
        for dir in dirs:
            cache = []
            ipc = []
            ipc_window = []
            cache_window= []
            for filename in os.listdir(os.path.join(subdir, dir)):
                f = os.path.join(subdir, dir, filename)
                actualfile = open(f,'r')
                if (filename[:3] == "ipc"):
                    ipc = actualfile.readlines()
                else:
                    cache = actualfile.readlines()
            dipc = window_size-len(ipc)
            dcache = window_size-len(cache)
            if dipc > 0:
                for x in range(dipc):
                    ipc.append('0')
            if dcache > 0:
                for x in range(dcache):
                    cache.append('0')
            if (checkNotEmpty(cache) and checkNotEmpty(ipc)):
                for x in range(0, len(ipc)):
                    ipc_window.append(float(ipc[x]))
                for x in range(0, len(cache)):
                    # let's normalize cache access to IPC
                    cache_window.append(float(cache[x])/1000000)
                cache_windows.append(getAverage(cache_window))
                ipc_windows.append(getAverage(ipc_window))
                if verdict:
                    ver_windows.append(0)
                valid += 1
    print("Total windows attack/bechmarks = " + str(valid))
    if verdict:
        dataframe = pd.DataFrame(list(zip(ipc_windows, cache_windows, ver_windows)), columns=['avg_ipc', 'avg_cache', 'verdict'])
    else:
        dataframe = pd.DataFrame(list(zip(ipc_windows, cache_windows)), columns=['avg_ipc', 'avg_cache']) 
    return dataframe


def create_reference(df, label):
    data = []
    for x in df.index:
        data.append(label)
    return data

def false_negative_rate(reference, prediction):
  false_negatives = 0
  true_positives = 0
  
  for i in range(len(reference)):
    if reference[i] == -1 and prediction[i] == 1:
      false_negatives += 1
    if reference[i] == -1:
      true_positives += 1
      
  false_negative_rate = false_negatives / (true_positives)
  print("False negatives = " + str(false_negatives))
  print ("Total positives =  " + str(true_positives))
  return false_negative_rate

def false_positive_rate(reference, prediction):
  #false negatives / positives
  false_positives = 0
  true_negatives = 0
  
  for i in range(len(reference)):
    if reference[i] == 1 and prediction[i] == -1:
      false_positives += 1
    if reference[i] == 1:
      true_negatives += 1
      
  false_positive_rate = false_positives / (true_negatives)
  print("False positives = " + str(false_positives))
  print("Total negatives = " + str(true_negatives))
  return false_positive_rate


def accuracy(reference, prediction):
    total = len(prediction) + 1
    matches = 0
    for i in range(total - 1):
        if reference[i] == prediction[i]:
            matches += 1
    return matches/total



## Data frame construction

In [ ]:
#target = 'tinyexr'
#bench = '../results/benchmarks/'

target = 'notbench/allbench/fmm'
bench = "../results/notbench/notfmm/" # change me if necessary

normal = "../results/" + target  +"/normal/"
attack = "../results/" + target  +"/attack/"
#window_size = 50  #change me if necessary: 10 for custom, 50 for tiny and pdf
window_size = 300 # 300  radiosity, raytrace 300 (works), fmm(kinda), watern(play with wsize and sampling), barnes (crashes, discard possibles crashes)
verdict = False

df_normal = getWindowFrameFromDir(normal,window_size, verdict)
df_attack = getWindowFrameFromDirAttack(attack,window_size, verdict)
print("Bench: ")
df_bench  = getWindowFrameFromDirAttack(bench,window_size, verdict)

## Feature stuff

In [ ]:

#df_bench  = getCompleteWindowFrameFromDirAttack(bench,window_size)
#df_normal = getCompleteWindowFrameFromDir(normal,window_size)
#df_attack = getCompleteWindowFrameFromDirAttack(attack,window_size)

In [ ]:
abnormal_count =  len(df_normal.index) - len(df_attack.index)
full_df = pd.concat([df_normal, df_attack],ignore_index=True)
full_df

In [ ]:
# ar = full_df.to_numpy()

# hh = len(full_df.index)
# dim = 3

# w, h, l = dim, hh*window_size, window_size
# Matrix = [[0 for x in range(w)] for y in range(h)] 

# for int3 in range(h):
#   Matrix[int3][0] = (int3//l)
#   Matrix[int3][1] = ar[math.floor(int3//l)][int3%l]
#   Matrix[int3][2] = ar[math.floor(int3//l)][(int3%l)+l]



# #tags = []
# #for x in range (0, hh):
# #    tags.append(x)

# df = pd.DataFrame(Matrix, columns =['Tag', 'ipc', 'cache'])
# #df = pd.DataFrame(Matrix, columns =['Tag', 'cache'])
if (verdict):
    X = full_df.loc[:, full_df.columns!="verdict"]
    y = full_df["verdict"]
    y



In [ ]:
#X_new = mutual_info_classif(X,y)
#X_new

In [ ]:

#extraction_settings = ComprehensiveFCParameters()
#data_filtered = extract_relevant_features(df, y, column_id='Tag', default_fc_parameters=extraction_settings) 

In [ ]:
#data_filtered

In [ ]:
iter = 1 #change for time measurement
#import seaborn as sns
#sns.histplot(df_attack['avg_cache'], kde=True, stat='probability')
#sns.histplot(df_attack['avg_ipc'], kde=True, stat='probability')


## Writing normal and attack traces to file

In [ ]:
log = False
if (log):
    with open('../results/paper/normal', 'w') as f:
        dfAsString = np.log10(df_normal*1000000).to_string(header=True, index=False)
        f.write(dfAsString)

    with open('../results/paper/attack', 'w') as f:
        dfAsString = np.log10(df_attack*1000000).to_string(header=True, index=False)
        f.write(dfAsString)
else:
    with open('../results/paper/normal', 'w') as f:
        dfAsString = df_normal.to_string(header=True, index=False)
        f.write(dfAsString)

    with open('../results/paper/attack', 'w') as f:
        dfAsString = df_attack.to_string(header=True, index=False)
        f.write(dfAsString)

## Unsupervised stuff

### Dataset visualization (normal data)

In [ ]:
df_merged = pd.concat([df_normal], ignore_index=True)
x_axis = df_merged["avg_ipc"]  
y_axis = df_merged["avg_cache"]  

x_a_axis = df_attack["avg_ipc"]
y_a_axis = df_attack["avg_cache"]    


x_norm, x_anorm, y_norm, y_anorm = [], [], [], []
for i in range(len(df_normal.index)):
    x_norm.append(x_axis[i])
    y_norm.append(y_axis[i])
for i in range(len(df_attack.index)):
    x_anorm.append(x_a_axis[i])
    y_anorm.append(y_a_axis[i])

# Plotting
#plt.xlim(0,0.25) #nolimit for tinyexr
#plt.ylim(0,1) #2.5 for tinyexr, 1 for pcf2bdf
plt.xlabel("IPC")
plt.ylabel("Cache accesses (x10^6)")
c1 = plt.scatter( x_norm,y_norm, c='green', s = 1)
c2 = plt.scatter( x_anorm, y_anorm, c='magenta', s = 1)
plt.legend([c1, c2], ['normal', 'abnormal'])



#plt.scatter(y_axis, x_axis)
#plt.show()

### One-class SVM

O(n^2) complexity one-class support vector machine

In [ ]:
from sklearn import datasets
from sklearn import svm
from sklearn.svm import OneClassSVM
from sklearn.model_selection import train_test_split


X_train, X_eval = train_test_split(df_normal, shuffle = True,test_size=0.5)


clf = OneClassSVM(gamma='scale', kernel='rbf', nu=0.06).fit(X_train) #0.07
#TODO: explore nu value https://scikit-learn.org/stable/modules/generated/sklearn.svm.OneClassSVM.html#sklearn.svm.OneClassSVM.decision_function
Y_pred_attack = clf.predict(df_attack)
Y_pred_normal = clf.predict(X_eval)

start = time.time_ns()
dif = 0
for x in range(iter):
    start = time.time_ns()
    Y_pred_bench = clf.predict(df_bench)
    end = time.time_ns()
    dif += end - start
print("Time OSVM = " + str(dif/(iter*1000*len(Y_pred_bench))) + "us")

y_train_normal = create_reference(X_eval, 1)
y_train_attack = create_reference(df_attack, -1)
y_train_bench = create_reference(df_bench, -1)

#print(len(y_train_attack))

y_train = np.concatenate([y_train_normal, y_train_attack]) #y_train_bench y_train_attack
y_pred = np.concatenate([Y_pred_normal, Y_pred_attack]) #Y_pred_bench, Y_pred_attack
df_merged =  pd.concat([X_eval, df_attack, df_bench], ignore_index=True)
x_axis = df_merged["avg_ipc"]  
y_axis = df_merged["avg_cache"]  

x_norm, x_anorm, y_norm, y_anorm = [], [], [], []
for i in range (len(y_pred)):
    if y_pred[i] == 1:
        x_norm.append(x_axis[i])
        y_norm.append(y_axis[i])
    else:
        x_anorm.append(x_axis[i])
        y_anorm.append(y_axis[i])

sc_osvm =  round(accuracy(y_train, y_pred)*100,2)
fn_osvm  = round(false_negative_rate(y_train, y_pred)*100,2)
print("[Full set] Accuracy = ", sc_osvm , "\t", round(false_positive_rate(y_train, y_pred)*100,2), "\t", fn_osvm)

# # Plotting
c1 = plt.scatter(y_norm, x_norm, c='green', s =1)
c2 = plt.scatter(y_anorm, x_anorm, c='magenta', s =1)
plt.legend([c1, c2], ['normal', 'abnormal'])
#plt.xlim(0,0.3) #nolimit for tinyexr
#plt.ylim(0,1) #2.5 for tinyexr, 1 for pcf2bdf
plt.xlabel("IPC")
plt.ylabel("Cache accesses (x10^6)")
plt.title(target +"  predictions One-Class SVM")
plt.show()
#plt.plot(y_train, c = 'magenta',  linestyle = 'None', marker='*')
#plt.plot(y_pred, c= 'cyan', linestyle='None', marker='3')
#plt.show()

### SGD-OSVM

Linear(-complexity) approximation to guassian-kernel One-Class SVM using Stochastic Gradient Descent

In [ ]:
from sklearn import linear_model

X_train, X_eval = train_test_split(df_normal, shuffle = True,test_size=0.5)
clf = linear_model.SGDOneClassSVM( nu=0.07)

clf.fit(X_train)

Y_pred_attack = clf.predict(df_attack)
Y_pred_normal = clf.predict(X_eval)
Y_pred_bench = clf.predict(df_bench)

y_train_normal = create_reference(X_eval, 1)
y_train_attack = create_reference(df_attack, -1)
y_train_bench = create_reference(df_bench, -1)


y_train = np.concatenate([y_train_normal, y_train_attack, y_train_bench])
y_pred = np.concatenate([Y_pred_normal, Y_pred_attack, Y_pred_bench])
df_merged =  pd.concat([X_eval, df_attack, df_bench], ignore_index=True)
x_axis = df_merged["avg_ipc"]  
y_axis = df_merged["avg_cache"]  

x_norm, x_anorm, y_norm, y_anorm = [], [], [], []
for i in range (len(y_pred)):
    if y_pred[i] == 1:
        x_norm.append(x_axis[i])
        y_norm.append(y_axis[i])
    else:
        x_anorm.append(x_axis[i])
        y_anorm.append(y_axis[i])
sc_sgd =  round(accuracy(y_train, y_pred)*100,2)
fn_sgd = round(false_negative_rate(y_train, y_pred)*100,2)
print("[Full set] Accuracy = ", sc_sgd , "\t", round(false_positive_rate(y_train, y_pred)*100,2), "\t", fn_sgd , "%")

# # Plotting
c1 = plt.scatter(y_norm, x_norm, c='green')
c2 = plt.scatter(y_anorm, x_anorm, c='magenta')
plt.legend([c1, c2], ['normal', 'abnormal'])
#plt.xlim(0,1) #nolimit for tinyexr
#plt.ylim(0,1) #2.5 for tinyexr, 1 for pcf2bdf
plt.xlabel("IPC")
plt.ylabel("Cache accesses (x10^6)")
plt.title(target +" predictions Stochastic Gradient Descend OSVM")
plt.show()
#plt.plot(y_train, c = 'magenta',  linestyle = 'None', marker='*')
#plt.plot(y_pred, c= 'cyan', linestyle='None', marker='3')
#plt.show()

### Local outlier factor (LOF)

In [ ]:
from sklearn import datasets
from sklearn import svm
from sklearn.neighbors import LocalOutlierFactor

X_train, X_eval = train_test_split(df_normal, shuffle = True,test_size=0.5)

lof = LocalOutlierFactor(novelty=True,n_neighbors=15) # change n_neighbors
lof.fit(X_train.values)

Y_pred_attack = lof.predict(df_attack)
Y_pred_normal = lof.predict(X_eval)
dif = 0
for x in range(iter):
    start = time.time_ns()
    Y_pred_bench = lof.predict(df_bench)
    end = time.time_ns()
    dif += end - start
print("Time LOF = " + str(dif/(iter*1000*len(Y_pred_bench))) + "us")


y_train_normal = create_reference(X_eval, 1)
y_train_attack = create_reference(df_attack, -1)
y_train_bench = create_reference(df_bench, -1)


y_train = np.concatenate([y_train_normal, y_train_attack])  #y_train_bench y_train_attack
y_pred = np.concatenate([Y_pred_normal, Y_pred_attack])  #Y_pred_bench, Y_pred_attack
df_merged =  pd.concat([X_eval, df_attack, df_bench], ignore_index=True)
x_axis = df_merged["avg_ipc"]  
y_axis = df_merged["avg_cache"]  

x_norm, x_anorm, y_norm, y_anorm = [], [], [], []
for i in range (len(y_pred)):
    if y_pred[i] == 1:
        x_norm.append(x_axis[i])
        y_norm.append(y_axis[i])
    else:
        x_anorm.append(x_axis[i])
        y_anorm.append(y_axis[i])
sc_lof = round(accuracy(y_train, y_pred)*100,2)
fn_lof = round(false_negative_rate(y_train, y_pred)*100,2)
print("[Full set] Accuracy = ",sc_lof , "\t", round(false_positive_rate(y_train, y_pred)*100,2), "\t", fn_lof)

# # Plotting
c1 = plt.scatter(y_norm, x_norm, c='green')
c2 = plt.scatter(y_anorm, x_anorm, c='magenta')
plt.legend([c1, c2], ['normal', 'abnormal'])
#plt.xlim(0,1) #nolimit for tinyexr
#plt.ylim(0,1) #2.5 for tinyexr, 1 for pcf2bdf
plt.xlabel("IPC")
plt.ylabel("Cache accesses (x10^6)")
plt.title(target +"  predictions Local Outlier Factor")
plt.show()

### Outlier detection (w synthetic anormalities )

In [ ]:
# Synthetic anomaly generation
X_train, X_eval = train_test_split(df_normal, shuffle = True,test_size=0.5)
anomaly_ratio = 0.1 #

std = X_train.std()
avg = X_train.mean()
max = X_train.max()
min = X_train.min()
scac = []
sipc = []


#Separating the feature are in  4 regions, excluding the normal area data
for i in range(int(len(X_train.values) * anomaly_ratio)//4):
    scac.append(random.uniform(0, avg["avg_cache"]+5*std["avg_cache"]))
    sipc.append(random.uniform(0, min["avg_ipc"]))
    scac.append(random.uniform(0, avg["avg_cache"]+5*std["avg_cache"]))
    sipc.append(random.uniform(max["avg_ipc"], avg["avg_ipc"]+5*std["avg_ipc"]))
    scac.append(random.uniform(0, min["avg_cache"]))
    sipc.append(random.uniform(0,  avg["avg_ipc"]+5*std["avg_ipc"]))
    scac.append(random.uniform(max["avg_cache"], avg["avg_cache"]+5*std["avg_cache"]))
    sipc.append(random.uniform(0, avg["avg_ipc"]+5*std["avg_ipc"]))

df_synth = pd.DataFrame(list(zip(sipc,scac)), columns=['avg_ipc', 'avg_cache'])


X_train_merged = pd.concat([df_synth, X_train], ignore_index = True)
X_train_merged

x_axis = X_train_merged["avg_ipc"]  
y_axis = X_train_merged["avg_cache"]  

x_norm, x_anorm, y_norm, y_anorm = [], [], [], []
for i in range (len(X_train_merged)):
    if i >= len(df_synth.values):
        x_norm.append(x_axis[i])
        y_norm.append(y_axis[i])
    else:
        x_anorm.append(x_axis[i])
        y_anorm.append(y_axis[i])
# # Plotting
c1 = plt.scatter(y_norm, x_norm, c='green')
c2 = plt.scatter(y_anorm, x_anorm, c='magenta')
plt.legend([c1, c2], ['normal', '(synth) abnormal'])
#plt.xlim(0,1) #nolimit for tinyexr
#plt.ylim(0,1) #2.5 for tinyexr, 1 for pcf2bdf
plt.title("Normal and synthetic data distribution")
plt.xlabel("IPC")
plt.show()


In [ ]:
X_train_merged

### Elliptic envelop

In [ ]:
from sklearn import datasets
from sklearn import svm
from sklearn.covariance import EllipticEnvelope



cov = EllipticEnvelope(contamination=anomaly_ratio).fit(X_train_merged)

Y_pred_attack = cov.predict(df_attack)
Y_pred_normal = cov.predict(X_eval)
Y_pred_bench = cov.predict(df_bench)


y_train_normal = create_reference(X_eval, 1)
y_train_attack = create_reference(df_attack, -1)
y_train_bench = create_reference(df_bench, -1)


y_train = np.concatenate([y_train_normal, y_train_attack, y_train_bench])
y_pred = np.concatenate([Y_pred_normal, Y_pred_attack, Y_pred_bench])
df_merged =  pd.concat([X_eval, df_attack, df_bench], ignore_index=True)
x_axis = df_merged["avg_ipc"]  
y_axis = df_merged["avg_cache"]  

x_norm, x_anorm, y_norm, y_anorm = [], [], [], []
for i in range (len(y_pred)):
    if y_pred[i] == 1:
        x_norm.append(x_axis[i])
        y_norm.append(y_axis[i])
    else:
        x_anorm.append(x_axis[i])
        y_anorm.append(y_axis[i])
sc_env = round(accuracy(y_train, y_pred)*100,2)
fn_env = round(false_negative_rate(y_train, y_pred)*100,2)
print("[Full set] Accuracy = ",sc_env , "%, fp = ", round(false_positive_rate(y_train, y_pred)*100,2), "%, fn = ", fn_env , "%")

# # Plotting
c1 = plt.scatter(y_norm, x_norm, c='green')
c2 = plt.scatter(y_anorm, x_anorm, c='magenta')
plt.legend([c1, c2], ['normal', 'abnormal'])
#plt.xlim(0,1) #nolimit for tinyexr
#plt.ylim(0,1) #2.5 for tinyexr, 1 for pcf2bdf
plt.xlabel("IPC")
plt.ylabel("Cache accesses (x10^6)")
plt.title(target +"  predictions Elliptic Envelop")
plt.show()

### Isolation forest

In [ ]:
from sklearn.ensemble import IsolationForest

clf = IsolationForest(n_estimators=15)
clf.fit(X_train_merged)
Y_pred_attack = clf.predict(df_attack)
Y_pred_normal = clf.predict(X_eval)
dif = 0
for x in range(iter):
    start = time.time_ns()
    Y_pred_bench = clf.predict(df_bench)
    end = time.time_ns()
    dif += end - start
print("Time Isolation = " + str(dif/(iter*1000*len(Y_pred_bench))) + "us")


y_train_normal = create_reference(X_eval, 1)
y_train_attack = create_reference(df_attack, -1)
y_train_bench = create_reference(df_bench, -1)


y_train = np.concatenate([y_train_normal, y_train_attack]) #y_train_bench y_train_attack
y_pred = np.concatenate([Y_pred_normal, Y_pred_attack])#Y_pred_bench, Y_pred_attack
df_merged =  pd.concat([X_eval, df_attack, df_bench], ignore_index=True)
x_axis = df_merged["avg_ipc"]  
y_axis = df_merged["avg_cache"]  

x_norm, x_anorm, y_norm, y_anorm = [], [], [], []
for i in range (len(y_pred)):
    if y_pred[i] == 1:
        x_norm.append(x_axis[i])
        y_norm.append(y_axis[i])
    else:
        x_anorm.append(x_axis[i])
        y_anorm.append(y_axis[i])
sc_iso = round(accuracy(y_train, y_pred)*100,2)
fn_iso = round(false_negative_rate(y_train, y_pred)*100,2)
print("[Full set] Accuracy = ",sc_iso , "\t", round(false_positive_rate(y_train, y_pred)*100,2), "\t",fn_iso )

# # Plotting
c1 = plt.scatter(y_norm, x_norm, c='green')
c2 = plt.scatter(y_anorm, x_anorm, c='magenta')
plt.legend([c1, c2], ['normal', 'abnormal'])
#plt.xlim(0,1) #nolimit for tinyexr
#plt.ylim(0,1) #2.5 for tinyexr, 1 for pcf2bdf
plt.xlabel("IPC")
plt.ylabel("Cache accesses (x10^6)")
plt.title(target +"  predictions")
plt.show()
print (len(Y_pred_attack))


### Summary

In [ ]:
scores = [
          sc_osvm,
          sc_sgd,
          sc_lof,
          sc_env,
          sc_iso
          ]

algorithms = [
              "One-Class SVM",
              "SGD SVM",
              "Local Outlier Factor",
              "Elliptic Envelop",
              "IsolationForest"
              ]    

for i in range(len(algorithms)):
    print("The accuracy score achieved using "+algorithms[i]+" is: "+str(scores[i])+" %")

#sns.set(rc={'figure.figsize':(20,8)})

df = pd.DataFrame({'Algorithm':algorithms, 'Accuracy':scores})
sns.set(rc={'figure.figsize':(12,6)})
sns.barplot(data=df, x = 'Algorithm', y='Accuracy')
plt.ylim(0,100)
plt.show()

In [ ]:
scores = [
          fn_osvm,
          fn_sgd,
          fn_lof,
          fn_env,
          fn_iso
          ] 

for i in range(len(algorithms)):
    print("The false negative rate achieved using "+algorithms[i]+" is: "+str(scores[i])+" %")

sns.set(rc={'figure.figsize':(12,6)})

df = pd.DataFrame({'Algorithm':algorithms, 'False negative rate':scores})
sns.barplot(data=df, x = 'Algorithm', y='False negative rate')
plt.ylim(0,100)
plt.show()